In [1]:
from QualtricsAPI.Setup import Credentials
from QualtricsAPI.XM import XMDirectory
from QualtricsAPI.XM import MailingList
from QualtricsAPI.Survey import Responses
import shutil
import os
import requests
import zipfile
import json
import io
import pandas as pd
import re
import openpyxl
pd.set_option('display.max_columns', None)

### Getting a survey response export

In [3]:
apiToken = 'HbeDqDB2kRbueZdGg9EhjMt2DDElVuvBxTR7HKhU'
dataCenter = 'syd1'
#Get Survey Responses (Updated)
kwargs = {"token": apiToken, "data_center": dataCenter}
Credentials().qualtrics_api_credentials(**kwargs)
# ['useLabels', 'includeLabelColumns', 'exportResponsesInProgress', 'limit', 'seenUnansweredRecode', 'multiselectSeenUnansweredRecode', 'includeDisplayOrder', 'startDate', 'endDate', 'timeZone']
df = Responses().get_survey_responses(survey="SV_881V4mXITZKpQTs", verify=None, useLabels=True)
df.drop(1, inplace=True)
display(df.head())
folder = "BOH1/EOY Restorative"
os.makedirs(folder, exist_ok=True)
df.to_excel(f"{folder}/data raw.xlsx", index=False)

,StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,RecipientFirstName,RecipientEmail,ExternalReference,LocationLatitude,LocationLongitude,DistributionChannel,UserLanguage,Date,Student Given Name,Student Family Name,Student ID,Examiners,532_1,532_2,532_3,532_4,532_5,532_6,532_7,532_8,532_9,532_10,532_11,532_12,532_13,CI occurred,Q2.6_1,Q2.6_7,Q2.6_8,Q2.6_9,Q2.6_10,CI explanation,TM Scale,Entrustment,Q3.4,Q3.6_Id,Q3.6_Name,Q3.6_Size,Q3.6_Type,gname,fname,uomid,Station Score,circuit
0,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,Recipient First Name,Recipient Email,External Data Reference,Location Latitude,Location Longitude,Distribution Channel,User Language,Date,Student Given Name,Student Family Name,Student ID,Examiners,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,532 SIM Adhesive restoration – two surfaces – ...,Has a critical item occurred?,Reason for the critical item - Damage to adjac...,Reason for the critical item - Damage to asses...,Reason for the critical item - Excessive remov...,Reason for the critical item - Caries remainin...,Reason for the critical item - Excessive under...,Please record if there was another type of cri...,Time Management Scale - please use your profes...,Entrustment Scale - please use your profession...,Please provide written feedback for the student,Please sign this form - Id,Please sign this form - Name,Please sign this form - Size,Please sign this form - Type,gname,fname,uomid,Station Score,circuit
2,2024-10-31 00:44:03,2024-10-31 00:45:43,IP Address,128.250.0.25,56,100,False,2024-11-01 00:45:44,R_4wivcqW5gJjFM4h,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,30 Oct 2024,test,test,1222222,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exam
3,2024-10-31 05:19:04,2024-10-31 05:19:33,IP Address,120.151.68.167,38,29,False,2024-11-01 05:24:27,R_4Q3iSBrVlCRxaOz,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,30 Oct 2024,-,-,-,Yvonne Zhang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exam
4,2024-10-31 04:33:00,2024-10-31 05:54:19,IP Address,120.151.68.167,38,4879,False,2024-11-01 05:54:21,R_4BOZnc9dcYiA09u,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,30 Oct 2024,1,1,1,Abella Huynh,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exam
5,2024-10-31 05:20:18,2024-10-31 05:54:20,IP Address,120.151.68.167,56,2041,False,2024-11-01 05:54:22,R_4GfaOdbSf8baXp9,NaN,NaN,NaN,NaN,NaN,NaN,anonymous,EN,30 Oct 2024,1,1,1,Yvonne Zhang,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,exam


## Get list of surveys

In [5]:

apiToken = "HbeDqDB2kRbueZdGg9EhjMt2DDElVuvBxTR7HKhU"
surveyId = "SV_2gzosVh1heRGEwm"
fileFormat = "csv"
dataCenter = 'syd1'

# Setting static parameters
def getSurveyList(apiToken, dataCenter):
    baseUrl = "https://{0}.qualtrics.com/API/v3/surveys".format(dataCenter)
    headers = {
        "content-type": "application/json",
        "x-api-token": apiToken,
        }
    response = requests.request("GET", baseUrl, headers=headers)
    surveyList = response.json()['result']['elements']
    return surveyList

# get name id mapping
def getsurveyNameIdDict(surveyList):
    surveyNameIdDict = {}
    for survey in surveyList:
        surveyNameIdDict[survey['name']] = survey['id']
    return surveyNameIdDict

surveyList = getSurveyList(apiToken, dataCenter)
surveyNameIdDict = getsurveyNameIdDict(surveyList)
print(surveyNameIdDict)

{'BOH2 OSCE Station 6': 'SV_00N9s1obrGfqF70', 'BOH1 OSCE Station 1': 'SV_08wXaUTiN9KIqyO', 'BOH2 OSCE Station 15': 'SV_0e3LErnfzJQF67k', '2024 BOH2 Audit SHY': 'SV_0HFgCx95CZOpXMO', 'BOH1 OSCE Station 7': 'SV_0qTAH2cGPTL2P7U', 'DDS2 Cons Dent Assessment - Exam 2 May': 'SV_0SuMHYshnySXHKe', 'BOH2 OSCE Station 10': 'SV_0T9UZzunf5srAKW', 'BOH2 OSCE Station 9': 'SV_1GprQ1JnQAGRidg', 'Clinical Evaluation November 2024': 'SV_1Rl99MTSLRPT9Ii', 'BOH2 OSCE Station 13': 'SV_23HnyTSFqPxZ2jc', 'BOH2 OSCE Station 1': 'SV_2gzosVh1heRGEwm', 'BOH2 Viva Voce Assessment 2024': 'SV_2mAhOTCd57Z7RxI', 'BOH1 OSCE Station 6': 'SV_3szQApTRfYQ6vhc', 'BOH1 OSCE Station 10': 'SV_4TKWuafm5oaFSoS', 'BOH2 OSCE Station 2': 'SV_55aK3mlUsV0VFfo', 'CAF v0.1': 'SV_55rB7g219imlgR8', 'BOH1 OSCE Station 11': 'SV_5q1Ly4Uy0hGdBMG', 'DDS2 Endo Assessment 2': 'SV_6zJ5KC6fHiQwsw6', 'BOH2 OSCE Station 5': 'SV_80UCENemAZj2NOm', 'BOH2 SIM Exo Assessment': 'SV_868NaG9gLOrgodM', 'BOH1 OSCE Station 3': 'SV_8cyerPb6NpyhoCG', 'BOH2 OSC

Filter out a specific type of surveys

In [ ]:
BOH = [survey for survey in surveyNameIdDict.keys() if 'BOH2 OSCE Station' in survey]
print(BOH)
BOH = sorted(BOH, key=lambda x: int(re.search(r'(Station \d+)', x).group(1).split(' ')[1]))
print(BOH)
savepath = 'BOH2\Audit\November 1'
os.makedirs(savepath, exist_ok=True)
for surveyName in BOH:
    surveyId = surveyNameIdDict[surveyName]
    surveyId = 'SV_0HFgCx95CZOpXMO'
    df = Responses().get_survey_responses(survey=surveyId, verify=None, useLabels=True)
    station = re.search(r'(Station \d+)', surveyName).group(1)
    print(station)
    df.drop(1, inplace=True)
    filename = f'{savepath}/All OSCE.xlsx'
    if not os.path.exists(filename):
        df.to_excel(filename, sheet_name= station, index=False)
    else:
        with pd.ExcelWriter(filename, engine = 'openpyxl', mode = 'a', if_sheet_exists='replace') as writer:
            df.to_excel(writer, sheet_name= station, index=False)
    print(surveyName)
    display(df.head())
    break